<h4>
  This code calculated weight of each word in the symptom list in source.csv and  mapped is to the label/disease. Then using the weight it predictes possible list of symptoms associated with a symptom liste extracted from a description (located in symptom-descreption.csv)
  <\h4>

In [1]:
#Clean the data from extract_symptom column and clean for apropriate encodign
def distill (sym):
    myt=[]
    #print (myt)
    sym=sym.split(",")
    #print (sym)
    for i in range (0,len(sym)):
        #remove characters
        sym[i]=sym[i].replace("_", " ")
        sym[i]=sym[i].replace("\'", "")
    #print(sym)
    #Remove words like in, from , of and convert string to a list of word 
    for i in sym:
        part_of_speech = get_part_of_speech(i)
        if part_of_speech != None and part_of_speech != "IN" and part_of_speech != "CC" :
            #print(f"The part of speech of '{word}' is: {part_of_speech}")
            lis=i.split(" ")
            #print (lis)
            if "" in lis:
                lis.remove("")
            if '' in lis:
                lis.remove('') 

            #print (lis)
            myt+=lis
    #print ("the list ", myt )
    return (myt)
# Function to get part of speech of a word
def get_part_of_speech(word):
    # Tokenize the word
    tokens = word_tokenize(word)
    # Get part of speech tags
    pos_tags = pos_tag(tokens)
    # Return the part of speech of the first token
    if len (pos_tags) >1:
        return pos_tags[0][1]
    else:
        return None

<h1>TF-IDF<h1>

In [2]:
import csv
import math
import argparse
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


#calculates the term weights for tokens in each document
#and saves in the token_by_doc dictionary
def calcwts(row_and_token, token_and_df):
    token_by_row = {}
    
    #iterates through the file in the dictionary
    for row in row_and_token:

        #iterates through the tokens in each file
        for token in row_and_token[row]:
            #checks if the token is in the list of tokens in the corpus
            if token in token_and_df:
                df = token_and_df[token]  #gets df of token
                tf = row_and_token[row][token] #gets tf of token       
                sum_tf = sum(row_and_token[row].values())  #totalfreq of all tokens 
                norm_tf = tf / sum_tf #normalize the tf

                #calculates the weight using the formula from the book
                weight = math.log10(1 + norm_tf) * math.log10(len(row_and_token) / df)

        
                #saves the weight for the token and the document id
                token_by_row[token] = token_by_row.get(token, {}) | {row : weight}  
    
    return token_by_row

    

def tokenize():
    disease_symptoms = {} #key:row number; value:list of symptoms for that row
    token_and_df = {} #key:symptom; value:list of rows where that symptom occur
    tokens_list = [] #all the symptoms

    with open("source.csv", mode="r") as file:  
        csvreader = csv.reader(file)
        next(csvreader, None) #ignores the first line
 
        count = 2 #initialize row number
        dis= " "
        for row in csvreader:
            dis= row[0]
            if dis in disease_symptoms:
                disease_symptoms[count] = distill(row[1])
                tokens_list += disease_symptoms[count]
            else:
                disease_symptoms[count] = distill(row[1])
                tokens_list += disease_symptoms[count]
                count+=1
            
    #iterates through each file
    for row in disease_symptoms:
        token_sum = {} #to save tokens and tf 

        #calculates the term frequency for the tokens
        for token in disease_symptoms[row]: 
                token_sum[token.strip()] = token_sum.get(token.strip(), 0) + 1
        
        disease_symptoms[row] = token_sum

        #saves the document frequency per term in the corpus
        for term in tokens_list:
            if term.strip() in token_sum:
                token_and_df[term.strip()] = token_and_df.get(term.strip(), 0) + 1
                
    #print(tokens_list)
    token_by_row = calcwts(disease_symptoms, token_and_df)
    #print (sorted(token_by_row['rash'].items(), key=lambda x:x[1], reverse=True))
    #print(token_by_row['headache'])
    #return retrieve(words, token_by_row)
    return token_by_row

#encode source.csv
token_by_row = tokenize()

In [25]:
#Predicts Top K disease given k, word(user symptom) and encoded token-weight dictionary
def retrieve(words, token_by_row,k):
    #print("retrebe")
    total_list=[]
    weightdict={}
    for word in words:
        part_of_speech = get_part_of_speech(word)
        if part_of_speech == "IN" or part_of_speech == "CC":
            words.remove(word)
    for word in words:
        #print(0,word)
        word=word.replace(" ","")
        if word in token_by_row:
            #print(1,word)
            count = 0
            for row, weight in sorted(token_by_row[word].items(), key=lambda x:x[1], reverse=True):
                if count > -1:
                    #print ("here", count)
                    df= pd.read_csv("source.csv")
                    #print( df.iloc[row-2, 0], weight)
                    if df.iloc[row-2, 0] not in total_list:
                        total_list.append(df.iloc[row-2, 0])
                        weightdict[df.iloc[row-2, 0]]= abs (weight)
                        #print ("list:", count, total_list)
                    else:
                        #print ("list:", count, total_list)
                        weightdict[df.iloc[row-2, 0]] += abs (weight)
                        #print ( weightdict[df.iloc[rw-2, 0]]+weight, weightdict[df.iloc[row-2, 0]])
                        
                #print (count)
                        count +=1
    weightdict={k: v for k, v in sorted(weightdict.items(), key=lambda item: item[1])}
    #print (weightdict)
    return list (weightdict.keys())[-k:]
        
  


In [11]:
import csv
tokens_list=[]
disease_symptoms = {} 
Truepostive=0
rank=[]
predicted_disease=[]

with open("symptom-description.csv", mode="r") as file:
        csvreader = csv.reader(file)
        next(csvreader, None) #ignores the first line
        count = 2 #initialize row number
        
        for row in csvreader:
            truevalue= row[0]
            #print ((distill(row[2])))
            result= retrieve(distill(row[2]), token_by_row,3)
            #print (result)
            for i in range (len(result)):
                if result[i].lower()== truevalue.lower():
                    #print ("true")
                    Truepostive+=1
                    rank.append(i)
                    predicted_disease.append(result[i])
            if (count > 30):
                break
            count+=1
print (Truepostive)
print (predicted_disease)
print (rank)


26
['acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne']
[2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1]


In [54]:
#Run it in parrallel of 32 batches for efficent use of the gpu
import pandas as pd
df= pd.read_csv("symptom-description.csv")
tokens_list=[]
disease_symptoms = {} 
Truepostive=0
rank=[]
predicted_disease=[]
batch_size= 50
inputs = df.iloc[:, 2].tolist()
truevalue= df.iloc[:, 0].tolist()
# Processing in batches
for i in range(1, 1150, batch_size):
    batch_inputs = inputs[i:i+batch_size]
    batch_tureval= truevalue[i:i+batch_size]
    result=[retrieve(distill(symp), token_by_row,5) for symp in batch_inputs]
   
    for i in range (len(result)):
        #print (i,result[i])
        for j in range (len(result[i])):
                if result[i][j] == batch_tureval[i].lower():
                    #print ("true")
                    Truepostive+=1
                    rank.append(j)
                    predicted_disease.append(result[i][j])
    #print ("ypred",predicted_disease)
    #print ("y" ,batch_tureval)
           
        
print (Truepostive)
print (predicted_disease)
print (rank)

  


412
['acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'allergy', 'allergy', 'allergy', 'allergy', 'allergy', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'common cold

In [55]:
#Run it in parrallel of 32 batches for efficent use of the gpu
import pandas as pd
df= pd.read_csv("symptom-description.csv")
tokens_list=[]
disease_symptoms = {} 
Truepostive3=0
rank=[]
predicted_disease=[]
batch_size= 50
inputs = df.iloc[:, 2].tolist()
truevalue= df.iloc[:, 0].tolist()
# Processing in batches
for i in range(1, 1150, batch_size):
    batch_inputs = inputs[i:i+batch_size]
    batch_tureval= truevalue[i:i+batch_size]
    result=[retrieve(distill(symp), token_by_row,3) for symp in batch_inputs]
   
    for i in range (len(result)):
        #print (i,result[i])
        for j in range (len(result[i])):
                if result[i][j] == batch_tureval[i].lower():
                    #print ("true")
                    Truepostive3+=1
                    rank.append(j)
                    predicted_disease.append(result[i][j])
    #print ("ypred",predicted_disease)
    #print ("y" ,batch_tureval)
           
        
print ("TOP 3:")
print (Truepostive3)
print (predicted_disease)
print (rank)

  


TOP 3:
369
['acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'allergy', 'allergy', 'allergy', 'allergy', 'allergy', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'chicken pox', 'common cold', 'common cold', 'co

In [56]:
#Run it in parrallel of 32 batches for efficent use of the gpu
import pandas as pd
df= pd.read_csv("symptom-description.csv")
tokens_list=[]
disease_symptoms = {} 
Truepostive10=0
rank=[]
predicted_disease=[]
batch_size= 50
inputs = df.iloc[:, 2].tolist()
truevalue= df.iloc[:, 0].tolist()
# Processing in batches
for i in range(1, 1150, batch_size):
    batch_inputs = inputs[i:i+batch_size]
    batch_tureval= truevalue[i:i+batch_size]
    result=[retrieve(distill(symp), token_by_row,10) for symp in batch_inputs]
   
    for i in range (len(result)):
        #print (i,result[i])
        for j in range (len(result[i])):
                if result[i][j] == batch_tureval[i].lower():
                    #print ("true")
                    Truepostive10+=1
                    rank.append(j)
                    predicted_disease.append(result[i][j])
    #print ("ypred",predicted_disease)
    #print ("y" ,batch_tureval)
           
print ("Top-10 ")        
print (Truepostive10)
print (predicted_disease)
print (rank)

  


Top-10 
551
['acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'allergy', 'allergy', 'allergy', 'allergy', 'allergy', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'arthritis', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'bronchial asthma', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', 'cervical spondylosis', '

In [44]:
#calculate accuraacy of difrent expermient
Truepostive10=551
Truepostive3=369
Truepostive=412
print ("TOP 10 accuracy is ",Truepostive10/1150, Truepostive10,Truepostive10-1150)
print ("TOP 5 accuracy is ",Truepostive/1150, Truepostive,Truepostive-1150)
print ("TOP 3 accuracy is ",Truepostive3/1150, Truepostive3,Truepostive3-1150)


TOP 10 accuracy is  0.4791304347826087 551 -599
TOP 5 accuracy is  0.3582608695652174 412 -738
TOP 3 accuracy is  0.3208695652173913 369 -781


In [30]:
import nltk
import csv
import math
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# Function definitions (tokenize, calcwts, retrieve, get_part_of_speech) go here...

def calculate_mrr(test_cases, ground_truth, token_by_row):
    reciprocal_ranks = []
    #print3 (test_cases[1])
    for [true_diseases, symptoms] in test_cases:
        predicted_diseases = retrieve(distill(symptoms), token_by_row,10)
        
        # Find the rank of the first relevant disease
        rank = 1
        found = False
        predicted_diseases.reverse()
        #print (predicted_diseases, true_diseases)
        for disease in predicted_diseases:
            if disease == true_diseases:
          #      print (disease, true_diseases)
           #     print (rank)
                reciprocal_ranks.append(1 / rank)
                found = True
                break
            rank += 1
        #print(reciprocal_ranks) 
        if not found:
            reciprocal_ranks.append(0)
    
    # Calculate the mean reciprocal rank
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
    return mrr


In [41]:
df= pd.read_csv("symptom-description.csv")
df = df.head(400)
data = [[row[0], row[2]] for row in df.itertuples(index=False, name=None)]
ground_truth = data[0:70]
test_cases =data[0:70]
# Calculate MRR
mrr = calculate_mrr(test_cases, ground_truth, token_by_row)
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")


Mean Reciprocal Rank (MRR): 0.4548
